In [1]:
import sys, time, re
import os
import pandas as pd
import pylab as plt
from importlib import reload
%matplotlib inline
import itertools as it
import numpy as np
from multiprocessing import Pool as ThreadPool
import  Levenshtein
from tqdm import tqdm_notebook as tqdm
import pickle
import gc
import sys
sys.path.append('../scripts')

import ml_utils as utils
reload(utils)

<module 'ml_utils' from '../scripts/ml_utils.py'>

In [21]:
ks_2samp([1,2,3], [1,2,3]).statistic

0.0

### Generate true mapping

In [170]:
utils.generate_true_mapping(from_raw=False)

          uid           uname          inst_name
0   751026638       mrzelkin1     aleksej zelkin
1     2744159         undruha  andrey gnelitskiy
2  4157858483       vadimbhai     abhairov vadim
3   389335890  superov_sergey     sergey superov
4   199238507       smilychka       nastja gogol
      uid           uname          vk_name
0    5394       mrzelkin1    alexey zelkin
1   22884         undruha  andre undrukhov
2   23754       vadimbhai    vadim reutsky
3   89831  superov_sergey   sergey superov
4  103177       smilychka     nastya gogol


'true_mapping.csv'

### Generate 2 step test data (data/generate/test2_predicted.pickle)

In [3]:
vk_1step, vk_others = utils.get_df_1step_and_others(suffix_name='vk')
inst_1step, inst_others = utils.get_df_1step_and_others(suffix_name='inst')
vk_others.head()

(26460, 3) (3000, 3) (16993, 3)
(24859, 3) (3000, 3) (16913, 3)


,uid,uname,vk_name
3723,48515973,gzalyaleeva,guzel zalyaleeva
3725,48537029,lisa.svit,alisa mukhametova
3726,48558965,fatkulik,evgenia fatkullina
3728,48588961,anastation_12,anastasia beloborodova
3729,48648949,vitya_siraev,vitya siraev


In [17]:
g1_fname = 'vk_lid_rid.csv'
g2_fname = 'inst_lid_rid.csv'
G1 = utils.read_g(g1_fname)
G2 = utils.read_g(g2_fname)

In [33]:
results, lid_rid = ml_iteration(vk_others, inst_others, forest, G1, G2, 
                    name_sim_threshold, threads = 6)

Start 0


HBox(children=(IntProgress(value=0, max=2832), HTML(value='')))

Start 1


HBox(children=(IntProgress(value=0, max=2832), HTML(value='')))

Start 2


...

HBox(children=(IntProgress(value=0, max=2832), HTML(value='')))

Start 5


HBox(children=(IntProgress(value=0, max=2833), HTML(value='')))


Thread 1 True = 315 False = 308 Total = 624 precision = 0.5048076923076923

Thread 0 True = 412 False = 347 Total = 760 precision = 0.5421052631578948

Thread 5 True = 289 False = 600 Total = 890 precision = 0.32471910112359553

Thread 2 True = 441 False = 501 Total = 943 precision = 0.4676564156945917

Thread 3 True = 404 False = 572 Total = 977 precision = 0.413510747185261

Thread 4 True = 445 False = 526 Total = 972 precision = 0.45781893004115226
0 true count 412
1 true count 315
2 true count 441
3 true count 404
4 true count 445
5 true count 289


98

In [98]:
utils.gen_train_data_2step(results, G1, G2)

HBox(children=(IntProgress(value=0, max=2156), HTML(value='')))

In [15]:
model2 = utils.retrain_model_step2(model = forest,
            step1_fname = 'step1_data.pickle', step2_fname='step2_data.pickle')

In [105]:
matched_1step, _ = utils.get_df_1step_and_others(count_1step=3300)

### Generate 3 step test data (data/generate/test3_data.pickle)

In [ ]:
def prepare_data_for_threads(df_l, df_r, model, G1, G2, threads, name_sim_threshold = None):
    data = []

    thr_size = len(df_l) // threads
    for i in range(threads):
        s = i * thr_size
        e = i * thr_size + thr_size
        d = {
            'thread_num': i,
            'vals_l': df_l[s:e] if i + 1 < threads else df_l[s:],
            'vals_r': df_r,
            'model': model,
            'G1': G1,
            'G2': G2,
        }
        if name_sim_threshold:
            d['name_sim_threshold'] =  name_sim_threshold
        data.append(d)
    return datareshape((1, -1)

In [175]:
reload(utils)

def collect_true(data):
    df_vals_l = data['vals_l']
    df_vals_r = data['vals_r']
    model = data['model']
    G1 = data['G1']
    G2 = data['G2']
    thread_num = data['thread_num']

    res_list = []
    lev = Levenshtein
    print('Start', thread_num)
    true_count = 0
    for row_l in tqdm(df_vals_l):
        for row_r in tqdm(df_vals_r):
#             ratio = lev.ratio(row_l[1], row_r[1])
            x = np.array(utils.feature(G1, row_l, G2, row_r)).reshape((1, -1))
            if model.predict(x) == 1:
                true_count += 1
                res_list.append((row_l, row_r)) # ratio
    total_count = len(df_vals_l) * len(df_vals_r)
    print('Thread', thread_num, 'True =', true_count, 'False =', total_count - true_count,
          'Total =', total_count, 'True/Total =', true_count / total_count)
    return res_list

def get_test_3_step_palalel(true_1step, true_2step, model, G1, G2, threads=4):
    
    utils.f_set1s = dict(pickle.load(open(os.path.join(utils.folder_gen, 'features_G1.pickle'), "rb")))
    utils.f_set2s = dict(pickle.load(open(os.path.join(utils.folder_gen, 'features_G2.pickle'), "rb")))
    filter_g = lambda G, col_num: list(set(G.nodes()) - (set((x[col_num] for x in true_1step)) | 
                                                      set((x[col_num] for x in true_2step))))
    df_l = filter_g(G1, 0)
    df_r = filter_g(G2, 1)
    data = utils.prepare_data_for_threads(df_l, df_r, forest, G1, G2, threads=threads)
    with ThreadPool(threads) as pool:
        results = pool.map(collect_true, data)
    439850
    for i, res in enumerate(results):
        print(i, 'true count', len(res))
    pickle.dump(results, open(os.path.join(utils.folder_gen, 'test3_predicted.pickle'), "wb"))
    utils.f_set1s = dict()
    utils.f_set2s = dict()
    return results

get_test_3_step_palalel(matched_1step, results2, forest, G1, G2, threads=5)
gc.collect()

Start 0


HBox(children=(IntProgress(value=0, max=4528), HTML(value='')))

HBox(children=(IntProgress(value=0, max=18638), HTML(value='')))

Start 1


HBox(children=(IntProgress(value=0, max=4528), HTML(value='')))

...

Start 4


HBox(children=(IntProgress(value=0, max=4532), HTML(value='')))

HBox(children=(IntProgress(value=0, max=18638), HTML(value='')))

Process ForkPoolWorker-100:
Process ForkPoolWorker-101:
Process ForkPoolWorker-102:
Traceback (most recent call last):
  File "/home/ildar/anaconda3/envs/ds/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ildar/anaconda3/envs/ds/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ildar/anaconda3/envs/ds/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ildar/anaconda3/envs/ds/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/home/ildar/anaconda3/envs/ds/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/home/ildar/anaconda3/envs/ds/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-175-03d90f68b960>", line 19, in 

KeyboardInterrupt: 

### Name threshold 82

In [36]:
lid_rid_90 = pickle.load(open(os.path.join(utils.folder_gen, 'lid_rid_90.pickle'), "rb"))
# ToDo retrain model there
df_l_others_80, df_r_others_80 = filter_others_from_predicted(vk_others, inst_others, lid_rid_90)

lid_rid_80 = ml_iteration(df_l_others, df_r_others, forest, G1, G2, 
                    name_sim_threshold = 0.8, threads = 6, save_to = 'lid_rid_80')

gc.collect()

Start 0


HBox(children=(IntProgress(value=0, max=2472), HTML(value='')))

Start 1


HBox(children=(IntProgress(value=0, max=2472), HTML(value='')))

Start 2


...

Start 4


HBox(children=(IntProgress(value=0, max=2472), HTML(value='')))

Start 5


HBox(children=(IntProgress(value=0, max=2477), HTML(value='')))


Thread 1 True = 503 False = 1729 Total = 2233 True/Total = 0.22525750111957007

Thread 0 True = 488 False = 1749 Total = 2238 True/Total = 0.21805183199285075

Thread 2 True = 409 False = 1691 Total = 2101 True/Total = 0.19466920514040933

Thread 5 True = 299 False = 1871 Total = 2171 True/Total = 0.1377245508982036

Thread 4 True = 465 False = 1786 Total = 2252 True/Total = 0.20648312611012434

Thread 3 True = 374 False = 1984 Total = 2359 True/Total = 0.15854175498092413
0 true count 488
1 true count 503
2 true count 409
3 true count 374
4 true count 465
5 true count 299
resulting 2 step 1322 from 2538  |  0.5208825847123719
lid_rid 1322 1322 1322


In [ ]:
utils.gen_train_data(lid_rid_80, G1, G2, save_to = 'train_data_80')

In [39]:
utils.precision(lid_rid_80)

0.6218434343434344


In [ ]:
from scipy.stats import ks_2samp

def dist_sim(G1, n, G2, m):
    sl = [v.degree() for v in vl.neighbors()]
    sr = [v.degree() for v in vr.neighbors()]
    return ks_2samp(sl, sr)

### Name threashold 70

In [55]:
lid_rid_80 = pickle.load(open(os.path.join(utils.folder_gen, 'lid_rid_80.pickle'), "rb"))

model_70 = utils.retrain_model(model_80, ['train_data_matched', 'train_data_90', 'train_data_80', ])

df_l_others_70, df_r_others_70 = utils.filter_others_from_predicted(df_l_others_80, df_r_others_80, lid_rid_80)

lid_rid_70 = utils.ml_iteration(df_l_others_70, df_r_others_70, model_70, G1, G2, 
                    name_sim_threshold = 0.7, threads = 6, save_to = 'lid_rid_70')

gc.collect()

14837 13253 1584 14837
Start 0


HBox(children=(IntProgress(value=0, max=2208), HTML(value='')))

Start 1


HBox(children=(IntProgress(value=0, max=2208), HTML(value='')))

Start 2


...

HBox(children=(IntProgress(value=0, max=2208), HTML(value='')))

Start 5


HBox(children=(IntProgress(value=0, max=2213), HTML(value='')))


Thread 4 True = 1285 False = 7939 Total = 9225 True/Total = 0.13929539295392954

Thread 0 True = 1720 False = 7947 Total = 9668 True/Total = 0.17790649565577163

Thread 5 True = 1393 False = 8357 Total = 9751 True/Total = 0.14285714285714285

Thread 3 True = 1478 False = 8398 Total = 9877 True/Total = 0.14964057912321554

Thread 1 True = 1787 False = 8484 Total = 10272 True/Total = 0.17396806853582555

Thread 2 True = 2247 False = 9812 Total = 12060 True/Total = 0.186318407960199
0 true count 1720
1 true count 1787
2 true count 2247
3 true count 1478
4 true count 1285
5 true count 1393
resulting 2 step 1925 from 9910  |  0.19424823410696265
lid_rid 1925 1925 1925


29

In [57]:
utils.gen_train_data(lid_rid_70, G1, G2, save_to = 'train_data_70')

HBox(children=(IntProgress(value=0, max=1925), HTML(value='')))

In [58]:
utils.precision(lid_rid_70)

0.2187012987012987


[[5394, 751026638],
 [22884, 2744159],
 [89831, 389335890],
 [103177, 199238507],
 [128184, 2680807187],
 [287383, 4355999142],
 [323426, 262205325],
 [396916, 2318740468],
 [411887, 324637050],
 [424983, 234009055],
 [440401, 1460125089],
 [460592, 963814117],
 [548294, 2008029075],
 [602277, 34259567],
 [625674, 252673120],
 [678747, 44827282],
 [688773, 1552256479],
 [690076, 174473229],
 [716194, 277655428],
 [723868, 705916151],
 [724031, 833911963],
 [730048, 42753460],
 [731235, 144453303],
 [733023, 332707352],
 [733627, 285052643],
 [738859, 1814863382],
 [763786, 1758978298],
 [771450, 1101678048],
 [786268, 535726771],
 [801855, 2349650083],
 [810285, 3296369720],
 [816936, 483551402],
 [851300, 278789282],
 [852042, 1517726154],
 [863837, 1212535796],
 [864321, 1965348809],
 [879258, 2272150903],
 [886873, 199731627],
 [929031, 1292158969],
 [960400, 1638475028],
 [1010702, 143723070],
 [1027431, 201851893],
 [1047557, 287870298],
 [1052030, 464674258],
 [1076944, 217836500

In [81]:
def overall_precision():
    ls = []
    for t in (6,7,8,9):
        lid_rid = pickle.load(open(os.path.join(utils.folder_gen, 'lid_rid_%d0.pickle' % t), "rb"))
        ls += lid_rid
        
    ls += [(x[0], x[1]) for x in pd.read_csv(os.path.join(utils.folder_gen, 'true_mapping.csv'))[:2500].values.tolist()]
    utils.precision(ls)
    
overall_precision()

0.6061955469506293


#### Features cache

In [74]:
utils.f_set1s = dict(pickle.load(open(os.path.join(utils.folder_gen, 'features_G1.pickle'), "rb")))
utils.f_set2s = dict(pickle.load(open(os.path.join(utils.folder_gen, 'features_G2.pickle'), "rb")))

### Name threashold 60

In [75]:
lid_rid_70 = pickle.load(open(os.path.join(utils.folder_gen, 'lid_rid_70.pickle'), "rb"))

model_60 = utils.retrain_model(model_70, ['train_data_matched', 'train_data_90', 'train_data_80', 'train_data_70'])

df_l_others_60, df_r_others_60 = utils.filter_others_from_predicted(df_l_others_70, df_r_others_70, lid_rid_70)

lid_rid_60 = utils.ml_iteration(df_l_others_60, df_r_others_60, model_60, G1, G2, 
                    name_sim_threshold = 0.6, threads = 6, save_to = 'lid_rid_60')

gc.collect()

13253 11328 1925 13253
Start 0


HBox(children=(IntProgress(value=0, max=1888), HTML(value='')))

Start 1


HBox(children=(IntProgress(value=0, max=1888), HTML(value='')))

Start 2


...

HBox(children=(IntProgress(value=0, max=1888), HTML(value='')))

Start 5


HBox(children=(IntProgress(value=0, max=1888), HTML(value='')))


Thread 4 True = 3100 False = 19265 Total = 22366 True/Total = 0.13860323705624608

Thread 5 True = 3338 False = 19437 Total = 22776 True/Total = 0.14655778011942394

Thread 0 True = 4374 False = 18807 Total = 23182 True/Total = 0.18868087309119144

Thread 2 True = 4208 False = 19718 Total = 23927 True/Total = 0.175868265975676

Thread 3 True = 3730 False = 20574 Total = 24305 True/Total = 0.15346636494548446

Thread 1 True = 4798 False = 19892 Total = 24691 True/Total = 0.19432181766635617
0 true count 4374
1 true count 4798
2 true count 4208
3 true count 3730
4 true count 3100
5 true count 3338
resulting 2 step 2165 from 23548  |  0.09193986750467131
lid_rid 2165 2165 2165


0

In [76]:
utils.precision(lid_rid_60)

0.13394919168591224


### Feature generation for ML on 1 step

In [28]:
%%time

features, labels = utils.gen_train_data_1step('step1_data.pickle', from_raw=False)

CPU times: user 16 ms, sys: 8 ms, total: 24 ms
Wall time: 19.8 ms


In [12]:
for a,b in it.islice(zip(features, labels), 2):
    print(a, b)

[30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 6, 1, 4, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.7666666666666667] 1
[30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 6, 1, 4, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 7, 2, 6, 0, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1082, 487, 233, 160, 85, 52, 31, 22, 12, 7, 6, 4, 0, 2, 0, 1, 1, 1, 0, 0, 1, 0.03333333333333333] 0


### ML models

In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

In [29]:
# X_train, X_test, y_train, y_test = train_test_split(
#      features, labels, test_size=0.3, random_state=42)
X_train, y_train = features, labels

In [30]:
forest = RandomForestClassifier(n_estimators = 400, criterion ='gini',
                        bootstrap = True, max_features = 'auto' ,n_jobs=4)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [196]:
forest.score(X_test, y_test)

0.88586118251928025

In [6]:
gbc = GradientBoostingClassifier(n_estimators=100)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [123]:
gbc.score(X_test, y_test)

0.72493573264781486

In [109]:
y_score = forest.predict(X_test)
y_true = y_test

fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',
label='GS, AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

NameError: name 'X_test' is not defined

### Choose string measurement method

In [9]:
from difflib import SequenceMatcher
import  Levenshtein
import itertools  as it
import time
    
seq = SequenceMatcher()
for vrow in it.islice(vk_1step.values, 10):
    print('-----------------')
    row = inst_1step[inst_1step['uname'] == vrow[1]]
    inst_name = row['inst_name'].values[0]
    vk_name = vrow[2]
    
    s = time.time()
    seq.set_seqs(vk_name, inst_name)
    a = seq.ratio()
    e = time.time()
    print('time 1:', e - s)
    b = Levenshtein.ratio(vk_name, inst_name)
    s = time.time()
    print('time 2:', s - e)
#     vector1 = text_to_vector(vk_name)
#     vector2 = text_to_vector(inst_name)
#     c = get_cosine(vector1, vector2)
#     print('time 3:', time.time() - s)
    print(vrow[0], row['uid'].values[0])
    print(vk_name, inst_name, a, b)

-----------------
time 1: 6.341934204101562e-05
time 2: 0.00027489662170410156
5394 751026638
alexey zelkin aleksej zelkin 0.8148148148148148 0.8148148148148148
-----------------
time 1: 5.817413330078125e-05
time 2: 0.000629425048828125
22884 2744159
andre undrukhov andrey gnelitskiy 0.5 0.5
-----------------
time 1: 3.0040740966796875e-05
time 2: 0.0001430511474609375
23754 4157858483
vadim reutsky abhairov vadim 0.37037037037037035 0.37037037037037035
-----------------
time 1: 3.2901763916015625e-05
time 2: 0.00012636184692382812
89831 389335890
sergey superov sergey superov 1.0 1.0
-----------------
time 1: 3.981590270996094e-05
time 2: 8.916854858398438e-05
103177 199238507
nastya gogol nastja gogol 0.9166666666666666 0.9166666666666666
-----------------
time 1: 3.838539123535156e-05
time 2: 8.416175842285156e-05
110208 1373668106
ilnur shaydullin none 0.2 0.2
-----------------
time 1: 6.29425048828125e-05
time 2: 8.630752563476562e-05
128184 2680807187
adelina ashrapova adelina f